In [3]:
# 필요한 라이브러리를 설치합니다.
! pip install segmentation_models_pytorch
! pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=aabdf5b7509f6e5dd0b5117f7376bd68e57aea924565739029d86468b678a8af
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=4103a909dc2c0330fdfc9df0a6f55c38308b06ff730eac921e8031bdec996202
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 필요한 라이브러리들을 임포트합니다.
from glob import glob
import os
# Log in to your W&B account
import wandb
import segmentation_models_pytorch as smp

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
TRAIN_NAME = "test"

### 데이터 경로를 설정합니다.

In [5]:
# 데이터 경로를 설정해줍니다.
data_directory = '/content/drive/MyDrive/Crack_Segmentation/dataset'

# 학습용 데이터 경로와 마스크 경로를 설정해줍니다.
train_image_directory = os.path.join(data_directory, 'train', 'images') # 학습용 이미지 경로
train_mask_directory = os.path.join(data_directory, 'train', 'masks') # 학습용 마스크 경로

# 테스트용 데이터 경로와 마스크 경로를 설정해줍니다.
test_image_directory = os.path.join(data_directory, 'test', 'images') # 테스트용 이미지 경로

In [6]:
# 각 경로에 존재하는 파일들의 경로를 리스트로 저장합니다.
train_image_paths = sorted(glob(os.path.join(train_image_directory, '*.jpg')))
train_mask_paths = sorted(glob(os.path.join(train_mask_directory, '*.png')))

test_image_paths = sorted(glob(os.path.join(test_image_directory, '*.jpg')))

# 각 이미지와 마스크의 수량을 체크하는 코드입니다.
print(f'학습용 이미지 개수 : {len(train_image_paths)}, 마스크 개수 : {len(train_mask_paths)}')
print(f'평가용 이미지 개수 : {len(test_image_paths)}')

학습용 이미지 개수 : 5986, 마스크 개수 : 5986
평가용 이미지 개수 : 3992


In [9]:
# 이미지와 마스크의 경로 예시를 보고 혹시 파일 리스트가 순서대로 정렬되어 있지 않는지 확인해봅니다.
print(f'이미지 경로 예시 : {train_image_paths[:5]}')
print(f'마스크 경로 예시 : {train_mask_paths[:5]}')

이미지 경로 예시 : ['/content/drive/MyDrive/Crack_Segmentation/dataset/train/images/train_00000.jpg', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/images/train_00001.jpg', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/images/train_00002.jpg', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/images/train_00003.jpg', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/images/train_00004.jpg']
마스크 경로 예시 : ['/content/drive/MyDrive/Crack_Segmentation/dataset/train/masks/train_00000.png', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/masks/train_00001.png', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/masks/train_00002.png', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/masks/train_00003.png', '/content/drive/MyDrive/Crack_Segmentation/dataset/train/masks/train_00004.png']


### Pytorch Dataset 클래스 정의
Pytorch Dataset 클래스는 AI 모델에 데이터를 입력하기 전에 데이터를 가공 및 표준화하는 역할을 합니다.

In [9]:
# 필요한 라이브러리들을 임포트합니다.
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torchvision import transforms

In [11]:
class SegDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, resize=(256, 256), mode='train'):
        '''
        image_paths: 이미지 경로들의 리스트
        mask_paths: 마스크 경로들의 리스트
        size: 이미지와 마스크를 리사이즈를 몇으로 할 지 결정하는 변수
        mode: train인지 test인지를 결정하는 변수
        '''
        self.image_paths = image_paths # 이미지 경로들의 리스트를 저장합니다.
        self.mask_paths = mask_paths # 마스크 경로들의 리스트를 저장합니다.
        self.resize = resize # 리사이즈 변수를 저장합니다.
        self.mode = mode # 모드 변수를 저장합니다.
        self.transform = transforms.Compose([
                transforms.ToTensor(), # 텐서 변환
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화
            ]) # 이미지를 텐서로 변환하고 정규화하는 코드입니다.

    def __len__(self):
        return len(self.image_paths) # 데이터셋의 길이를 반환합니다.

    def __getitem__(self, idx):
        image_path = self.image_paths[idx] # idx 번째 이미지 경로를 불러옵니다. ex) 전체 이미지 중 첫 번째 이미지
        filename = os.path.basename(image_path) # 이미지 경로에서 이미지 파일의 이름을 불러옵니다. ex) train_0001.png

        # image를 불러옵니다.
        image = Image.open(image_path) # 이미지 경로를 참조해 이미지를 불러옵니다. BGR 형태의 이미지입니다.
        image = image.convert('RGB') # RGB 형태로 변환합니다.
        image_size = torch.tensor(image.size) # 이미지의 사이즈를 저장합니다.

        # image를 resize합니다.
        image = image.resize(self.resize)

        # image를 torch tensor로 변환합니다.
        image = self.transform(image).float() # 딥러닝 모델의 입력은 float형이어야 합니다.

        if self.mode == 'train' or self.mode == 'val':
            mask_path = self.mask_paths[idx] # idx 번째 마스크 경로를 불러옵니다.

            # mask를 로드합니다.
            mask = Image.open(mask_path) # 마스크 경로를 참조해 마스크(정답)를 불러옵니다. Gray 이미지이며, 0부터 59 사이의 값을 가집니다.

            # mask를 resize합니다.
            # 마스크 이미지는 resize 할때 iterpolation 방식에 각별히 주의해야 합니다.
            # linear interpolation 등을 사용하면 유효하지 않은 mask 값이 될 수 있습니다.
            # 따라서 주변에 있는 픽셀 값으로 interpolation 해야합니다.
            mask = mask.resize(self.resize, resample=Image.NEAREST) # 마스크를 리사이징합니다.

            # mask를 torch tensor로 변환합니다.
            mask = torch.from_numpy(np.array(mask)).long() # 정답은 정수형이어야 하므로 long형으로 변환합니다.

        else:
            mask = np.zeros(self.resize) # 테스트 데이터셋의 경우 마스크가 없으므로 0으로 채워진 마스크를 만듭니다.
        # 이미지, 마스크, 파일이름, 이미지 원본 사이즈 순으로 반환합니다.
        return image, mask, filename, image_size

###  모델 학습과 검증을 수행합니다.

In [15]:
import shutil

# 학습된 모델을 저장할 경로를 설정합니다.
save_directory = f'/content/drive/MyDrive/Crack_Segmentation/results/{TRAIN_NAME}'


# 경로를 생성합니다.
os.makedirs(save_directory, exist_ok=True)
os.makedirs(save_directory+'/model', exist_ok=True)
os.makedirs(save_directory+'/pred', exist_ok=True)

In [13]:
from sklearn.model_selection import KFold
import torch.optim as optim

split = 2
kfold = KFold(n_splits=split, shuffle=False)

# 어떤 장치에서 학습할지 결정합니다. cuda는 gpu를 사용한다는 것을 의미합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [18]:
def calc_iou(mask, pred_mask): # assumes mask, pred_mask are in batch(that is, shape = [B, H, W])
    # calculates iou only for value 1(crack)
    inter_mask = mask * pred_mask
    inter_mask_sum = inter_mask.sum(axis = 0)
    mask_sum = mask.sum(axis = 0)
    pred_mask_sum = pred_mask.sum(axis = 0)
    iou_list = inter_mask_sum / (mask_sum + pred_mask_sum - inter_mask_sum) # n(A \Union B) = n(A) + n(B) - n(A \Intersection B)
    print(iou_list)
    return iou_list.mean()



In [19]:
A = torch.Tensor([
    [
        [1, 1],
        [1, 0]
    ],
    [
        [1, 0],
        [0, 0]
    ]
])
B = torch.Tensor([
    [
        [1, 1],
        [1, 0]
    ],
    [
        [1, 1],
        [1, 0]
    ]
])
calc_iou(A, B)



tensor([[1.0000, 0.5000],
        [0.5000,    nan]])


tensor(nan)

In [16]:
# 필요한 라이브러리를 임포트합니다.
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

num_epochs = 1  #학습할 에폭의 수를 결정합니다.
for i, (train_index, test_index) in enumerate(kfold.split(train_image_paths, train_mask_paths)):
    # WandB 사용을 위한 초기 설정을 합니다.
    wandb.init(
        project="crack_segmentation",
        name=f"{TRAIN_NAME}fold_{i}",
        config={
            "encoder_name" : 'timm-efficientnet-b0',
            'encoder_weights' : 'imagenet',
            "learning_rate": 0.02,
            "epochs": 10,
            "batch_size": 16,
            'learning_rate':0.01
        })

    config = wandb.config
    # 데이터 split

    fold_train_image_paths = np.array(train_image_paths)[train_index].tolist()[:100]
    fold_train_mask_paths = np.array(train_mask_paths)[train_index].tolist()[:100]
    fold_valid_image_paths = np.array(train_image_paths)[test_index].tolist()[:100]
    fold_valid_mask_paths = np.array(train_mask_paths)[test_index].tolist()[:100]

    # 데이터셋 & 데이터 로더 생성
    train_dataset = SegDataset(fold_train_image_paths, fold_train_mask_paths, mode='train') # 학습 (train) 데이터 인스턴스 생성
    val_dataset = SegDataset(fold_valid_image_paths, fold_valid_mask_paths, mode='val') # 검증 (valid) 데이터 인스턴스 생성
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=2) # 학습 데이터 로더
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=2) # 검증 데이터 로더

    # 모델 & optimizer & loss function 설정
    model = smp.Unet(classes=2,
                    encoder_name = config['encoder_name'],
                    encoder_weights = config['encoder_weights']).to(device)

    optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])

    loss_function = nn.CrossEntropyLoss()

    best_iou = 0

    for epoch in range(num_epochs):

        # 학습을 수행합니다.
        model.train() # 모델을 학습 모드로 설정합니다.


        # 학습 데이터에 대한 정답율을 기록할 리스트를 선언합니다.
        train_losses = []

        # 학습 데이터셋으로부터 데이터를 가져옵니다.
        for image, mask, filename, image_size in tqdm(train_loader, position=0, leave=True, desc=f'Fold : {i} epoch: {epoch} | train'):

            # 가져온 데이터를 장치에 할당합니다.
            image = image.to(device) # 이미지의 차원은 [배치사이즈, 채널 수, 이미지 높이, 이미지 넓이]입니다.
            mask = mask.to(device) # 마스크의 차원은 [배치사이즈, 이미지 높이, 이미지 넓이]입니다.

            # 모델의 출력값을 계산합니다.
            # 예측 결과인 pred_mask는 [배치사이즈, 클래스 수, 이미지 높이, 이미지 넓이]차원의 행렬 형태를 가집니다.
            pred_mask = model(image)

            # loss를 계산합니다.
            loss = loss_function(pred_mask, mask)
            train_losses.append(loss.mean())

            # loss를 통해 모델을 학습합니다.
            optimizer.zero_grad() # gradient를 초기화합니다.
            loss.backward() # gradient를 계산합니다.
            optimizer.step() # gradient를 통해 파라미터를 업데이트합니다.

            # argmax 연산을 통해 확률이 가장 높은 클래스를 예측값으로 선택합니다.
            # [배치사이즈, 클래스 수, 이미지 높이, 이미지 넓이] -> [배치사이즈, 이미지 높이, 이미지 넓이]로 변환됩니다.
            pred_mask = torch.argmax(pred_mask, dim=1)

        # 검증을 수행합니다.
        model.eval() # 모델을 검증 모드로 설정합니다.

        # 검증 데이터에 대한 IOU를 기록할 리스트를 선언합니다.
        val_ious = []

        # 검증 데이터셋으로부터 데이터를 가져옵니다.
        for image, mask, filename, image_size in tqdm(val_loader, position=0, leave=True, desc=f'Fold : {i} epoch: {epoch} | val'):

                # 가져온 데이터를 장치에 할당합니다.
                image = image.to(device) # 이미지의 차원은 [배치사이즈, 채널 수, 이미지 높이, 이미지 넓이]입니다.
                mask = mask.to(device) # 마스크의 차원은 [배치사이즈, 이미지 높이, 이미지 넓이]입니다.

                # 모델의 출력값을 계산합니다.
                # 예측 결과인 pred_mask는 [배치사이즈, 클래스 수, 이미지 높이, 이미지 넓이]차원의 행렬 형태를 가집니다.
                pred_mask = model(image)

                # argmax 연산을 통해 확률이 가장 높은 클래스를 예측값으로 선택합니다.
                # [배치사이즈, 클래스 수, 이미지 높이, 이미지 넓이] -> [배치사이즈, 이미지 높이, 이미지 넓이]로 변환됩니다.
                pred_mask = torch.argmax(pred_mask, dim=1)

                # 해당 배치에 대한 성능 평가를 진행합니다.
                batch_iou =  #TODO

                val_ious.append(batch_iou)

        train_loss = sum(train_losses) / len(train_loss)
        val_iou = sum(val_ious) / len(val_iou)

        wandb.log({'train_loss' : train_loss, 'val_iou': val_iou})

        print(f'Fold : {i}, 에폭 : {epoch}, 학습 loss : {train_loss}, 검증 정확도 : {val_iou}')

        # validation accuracy가 최고를 갱신하면 모델을 저장합니다.
        if val_iou > best_iou: # 이번 에폭의 기존의 최고 accuracy보다 높으면
            best_acc = val_iou # 최고 accuracy를 갱신하고
            torch.save(model.state_dict(), f'{save_directory}/model/best_model_fold_{i}.pt') # 모델을 저장합니다.
            print(f'Fold : {i}, Best performance at epoch {epoch} : {best_iou}')


train_acc,▁
val_acc,▁
train_acc,0.49447
val_acc,0.87029


Fold : 0 epoch: 0 | train: 100%|██████████| 7/7 [00:02<00:00,  3.44it/s]
Fold : 0 epoch: 0 | val: 100%|██████████| 7/7 [00:01<00:00,  5.41it/s]


Fold : 0, 에폭 : 0, 학습 정확도 : 0.8133746555873326, 검증 정확도 : 0.9237478801182338
Fold : 0, Best performance at epoch 0 : 0.9237478801182338


train_acc,▁
val_acc,▁
train_acc,0.81337
val_acc,0.92375


Fold : 1 epoch: 0 | train: 100%|██████████| 7/7 [00:02<00:00,  2.58it/s]
Fold : 1 epoch: 0 | val: 100%|██████████| 7/7 [00:02<00:00,  3.24it/s]


Fold : 1, 에폭 : 0, 학습 정확도 : 0.8012298856462751, 검증 정확도 : 0.9526435307094029
Fold : 1, Best performance at epoch 0 : 0.9526435307094029


# 추론을 수행합니다.

In [17]:
# 예측 결과를 저장할 경로를 생성합니다.
pred_save_directory = save_directory + "/pred"

# 경로를 생성합니다.
os.makedirs(pred_save_directory, exist_ok=True)

In [18]:
# 학습이 완료된 모델을 로드합니다.
model = smp.Unet(classes=2, # 의류 종류 수에 맞게 클래스를 결정합니다.
                 encoder_name = 'timm-efficientnet-b0', # encoder 모델 구조를 결정합니다.
                 encoder_weights = 'imagenet') # encoder 모델에 어떤 사전학습 모델을 적용할지 지정합니다.

model = model.to(device)

In [19]:
# 데이터셋 & 데이터로더
test_dataset = SegDataset(test_image_paths, mode='test') # 평가 (test) 데이터 인스턴스 생성
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2) # 평가 데이터 로더

In [20]:
for image, _, filename, image_size in tqdm(test_loader, position=0, leave=True, desc=f'Prediction'):
    # 가져온 데이터를 장치에 할당합니다.
    image = image.to(device)

    # 모델의 출력값을 계산합니다.
    for fold in range(split):
        model.load_state_dict(torch.load(f'{save_directory}/model/best_model_fold_{fold}.pt'))
        if fold == 0:
            pred_mask = model(image)
        else:
            pred_mask += model(image)

    # argmax 연산을 통해 확률이 가장 높은 클래스를 예측값으로 선택합니다.
    pred_mask = torch.argmax(pred_mask, dim=1)

    for i, a_pred_mask in enumerate(pred_mask):
        # pred_mask를 PIL image로 변환합니다.
        pred_mask_image = Image.fromarray(np.uint8(a_pred_mask.cpu().numpy()))

        # 원본 이미지의 크기로 resize합니다.
        pred_mask_image = pred_mask_image.resize(image_size[i], resample=Image.NEAREST)

        filename_ = filename[i].replace('.jpg', '.png')


        # 이미지를 저장합니다.
        pred_mask_image.save(f'{pred_save_directory}/{filename_}')

Prediction: 100%|██████████| 250/250 [07:00<00:00,  1.68s/it]


### 예측 결과물을 압축해서 제출합니다.

In [21]:
import zipfile

pred_files = glob(f'{pred_save_directory}/*.png')

# 압축을 수행합니다.
with zipfile.ZipFile(save_directory+'/out-of-fold.zip', 'w') as zip:
    for pred_file in pred_files:
        zip.write(pred_file, os.path.basename(pred_file))